# 実験1 CVSデータの解析

In [ ]:
require("./cumps");
function print(s) { console.log(s); }

In [ ]:
var num = pat("[0-9]+")

In [ ]:
num("123abc") // "123abc"を他の文字列に変えて試してみよ

In [ ]:
var alpha = pat("[a-z]+");

In [ ]:
alpha("abc123")

[課題1] 小数を切り出すパーサfnumをつくるにはpatどのような正規表現を与えればよいか？ 

In [ ]:
var fnum = pat("ここにどのような正規表現を与えればよいか？");

テスト例（これだけでは十分ではない．他の必要なテスト例は自分で考えよ．）

In [ ]:
fnum("123abc")

In [ ]:
fnum("123.456abc")

In [ ]:
fnum("123.abc")

In [ ]:
fnum("123.456.789abc")

パーサの直列接続

In [ ]:
var comma = w(",");

In [ ]:
var p = seq(num,comma,num,comma,num);

In [ ]:
p("12,3,456,78,9")

パーサの反復

In [ ]:
var p = moreThan0(num);

In [ ]:
p("12 3 456 78 9")

コンマを間にはさんだ繰り返し(「カツサンド」パターン）

In [ ]:
var p = seq(num, moreThan0(seq(comma,num)));

In [ ]:
p("12, 3,456,78, 9")

カツサンドパターンを短く書く専用関数（上の例[16]と同じことが，より完結に書ける）

In [ ]:
var p = sepBy(num,comma); // 上の例[18]を再び実行してみよ

コンマの読み捨て

In [ ]:
var p = sepBy(num,","); // 上の例[18]を再び実行してみよ

パーサの並列接続

In [ ]:
var p = oneOf(num,alpha);

In [ ]:
p("123abc")

In [ ]:
p("abc123")

オプションの指定

In [ ]:
var p = seq(opt(num),alpha);

In [ ]:
p("123abc123")

In [ ]:
p("abc123")

パーサの定義中に変数名が再帰的に出現してもよい．ただしダミーの定義が必要になる．

（例）正しく対応のついたかっこの並び
$E \rightarrow \varepsilon \ \mid \ ( E ) \ E$
（あるいは右辺に正規表現を流用して
$E \rightarrow ( E )*$）

In [ ]:
function pexp(s) { return pexp(s); } // ダミーの定義
var pexp = moreThan0(seq(w("("),pexp,w(")")));

In [ ]:
pexp("((())())")

In [ ]:
pexp("())(")

In [ ]:
pexp(")()(")

別解

In [ ]:
function pexp2(s) { return pexp2(s); } // ダミーの定義
var pexp2 = opt(seq(w("("),pexp2,w(")"),pexp2));

In [ ]:
pexp2("((())())")

[課題2] pexpあるいはpexp2を拡張して，丸かっこだけでなくブラケット（[,]）とブレース（{,}）も許すパーサxpexpを定義せよ．

In [ ]:
function xpexp(s){ return xpexp(s); }
// 以下自分で
// var xpexp = ...

In [ ]:
xpexp("{[()]()}")

In [ ]:
xpexp("(]")